# 466 final project

### What we want 

we want to be able to take a set of users and use their previous transactions to predict what dapps they will use in the future.

## Load and clean user account data
these users will be split up into training and testing sets
this list of users was pulled from pontem's space pirate whitelist.
Pontem is a dapp that allows users to earn 'interest' on their crypto holdings by providing liquidity through their dex (sets of liquidity pools).

In [80]:
import pandas as pd
#load the users csv file (
users = open('users.csv').readlines()
user_temp = []
for user in users:
    if len(user)>=66:
        user_temp.append(user.split(',')[0])
print("ALL user ADDR COUNT:",len(users))
users = user_temp

print("PROPPER ADDR COUNT:",len(users))

# Write the users to a new file
# with open('users_clean.csv', 'w') as f:
#     for user in users:
#         f.write(user)


ALL user ADDR COUNT: 31622
PROPPER ADDR COUNT: 31622


We also need to load in the information about the dapps that we want to predict for. We will use the dapp list created by 'seam-money' an explorer for dapps on the network

In [81]:
dapps = open('dapps.csv').readlines()
# print(dapps)
dapp_dic = {}
for dapp in dapps:
    d = dapp.split(',')
    dapp_dic[d[0]] = d[1]
print("DAPP COUNT:",len(dapp_dic))
print(dapp_dic)


DAPP COUNT: 14
{'0x34e2eead0aefbc3d0af13c0522be94b002658f4bef8e0740a21086d22236ad77': 'Switchboard', '0x62fdfe47c9c37227be1f885e79be827be292fe1833ac63a2fe2c2c16c55ecb12': 'KanaLabs', '0xd1fd99c1944b84d1670a2536417e997864ad12303d19eac725891691b04d614e': 'BlueMove', '0x89576037b3cc0b89645ea393a47787bb348272c76d6941c574b053672b848039': 'Hippo Labs', '0x2c7bccf7b31baf770fdbcc768d9e9cb3d87805e255355df5db32ac9a669010a2': 'Souffl3', '0x9770fa9c725cbd97eb50b2be5f7416efdfd1f1554beb0750d4dae4c64e860da3': 'Aries', '0x190d44266241744264b964a37b8f09863167a12d3e70cda39376cfb4e3561e12': 'liquidSwap', '0x16fe2df00ea7dde4a63409201f7f4e536bde7bb7335526a35d05111e68aa322c': 'AnimeSwap', '0xa61f1c8f0e69bb866f2073f9a1ab0252efdd9f12c092256c68290d8dae35551e': 'Mojito', '0xd4c70815e9d245a097646af821ecf87a073039e71e93c8cd04a0da082134d296': 'Topaz', '0x98298d34bcf896c663e069c464754e0cfd36b50e21eedd8db0e4189168057cb7': 'Argo', '0x7ab72b249ec24f76fe66b6de19dcee1e3d3361db5c2cccfaa48ea8659060a1bd': 'HoustonSwap', '0

In [82]:
users[1]

'0x6d19623096d04c409b180b936f23a47d48aa4c3a48ebc21909d65308b1c7ccee'

In [84]:
# from flask import jsonify
import requests
import random
NODE_URL = "https://fullnode.mainnet.aptoslabs.com/v1"
def get_account_transactions(address):
    user_dapps = {}
    response = requests.get(
        f"{NODE_URL}/accounts/{address}/transactions",
        params={"limit": 100}
        )
    txs = response.json()
    # print(len(txs))
    txns_temp = []
    for tx in txs:
        # print(tx['type'])
        try:
            txn = {}
            payload = tx['payload']
            to = payload['function'].split('::')[0]
            func = payload['function'].split('::')[2]
            mod = payload['function'].split('::')[1]
            txn['function'] = func
            txn['module'] = mod
            txn['sender'] = to
            txn['arguments'] = payload['arguments']
            txn['type_args'] = payload['type_args']
            txn['address'] = tx['sender']
            if to in dapp_dic.keys():
                # print(dapp_dic[to])
                dapp_name = dapp_dic[to]
                if dapp_name in user_dapps.keys():
                    user_dapps[dapp_name] = 1 + user_dapps[dapp_name]
                    
                else:
                    user_dapps[dapp_name] = 1
        except:
            # print(e)
            pass

    return txs,user_dapps


def find_dapps_used(address):
    transactions,dapps = get_account_transactions(address)
    return dapps


users_used_dapps = {}
demo_users = random.sample(users, 100)
for user in users[120:200]:
    # print("USER:",user)
    users_used_dapps[user] = find_dapps_used(user)

    print(users_used_dapps[user])
    # find_dapps_used(user)
# dapps = find_dapps_used(users[0])

# write the dapps to a new file
# with open('users_used_dapps.csv', 'w') as f:
#     for user in users_used_dapps.keys():
#         f.write(jsonify(users_used_dapps[user]))
    





{'liquidSwap': 1}
{'Hippo Labs': 2, 'liquidSwap': 4, 'Souffl3': 1, 'Topaz': 1}
{'BlueMove': 1, 'liquidSwap': 1, 'Souffl3': 3, 'Hippo Labs': 1}
{'Souffl3': 9, 'liquidSwap': 1, 'Topaz': 1}
{'liquidSwap': 5, 'Souffl3': 13, 'Topaz': 1}
{'liquidSwap': 10, 'AnimeSwap': 3, 'Aries': 2}
{'liquidSwap': 1, 'Souffl3': 1, 'BlueMove': 3}
{'liquidSwap': 7, 'Cetus': 4, 'Souffl3': 4, 'Topaz': 5, 'KanaLabs': 1}
{'liquidSwap': 1}
{'liquidSwap': 14, 'Souffl3': 1}
{'liquidSwap': 5, 'LayerZero': 1, 'Souffl3': 1}
{'Hippo Labs': 4, 'liquidSwap': 10}
{'liquidSwap': 3, 'Hippo Labs': 6}
{'BlueMove': 1, 'liquidSwap': 1}
{'Cetus': 7, 'liquidSwap': 6, 'Hippo Labs': 6}
{'Souffl3': 3, 'Topaz': 3, 'AnimeSwap': 1, 'BlueMove': 4}
{'liquidSwap': 1, 'Hippo Labs': 9, 'Cetus': 10}
{'liquidSwap': 25}
{'Souffl3': 4, 'BlueMove': 1}
{'liquidSwap': 18}
{'liquidSwap': 2, 'Souffl3': 3, 'Topaz': 4, 'Cetus': 1}
{'Argo': 5, 'liquidSwap': 1, 'LayerZero': 3, 'Souffl3': 1}
{'Argo': 8, 'liquidSwap': 3, 'Hippo Labs': 1, 'Souffl3': 2}
{'Hi

In [ ]:
FLASK_URL = "localhost:5000"